In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

import tensorflow as tf

In [2]:
!curl -L -o ../data/will_play_text.csv https://commondatastorage.googleapis.com/ckannet-storage/2012-04-24T183403/will_play_text.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9.8M  100  9.8M    0     0  5772k      0  0:00:01  0:00:01 --:--:-- 5770k


In [55]:
ç

In [56]:
df.head(10)

,row,piece,chapter,section,character,line
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil
8,9,Henry IV,1.0,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...
9,10,Henry IV,1.0,1.1.7,KING HENRY IV,"Nor more shall trenching war channel her fields,"


In [64]:
sentences = df[~df.character.isnull()].line

# mapping
letters = ['>'] + list(set(''.join(list(sentences))))
nletters = len(letters)

bw_mapping = dict(enumerate(letters))
fw_mapping = dict([(c, ix) for ix, c in enumerate(letters)])

sen_sz = 1 + 30

# dataset
sentences = [[fw_mapping[c] for c in ['>'] + list(s[:sen_sz - 1])] for s in sentences if len(s)>(sen_sz - 1)]
sentences = np.array(sentences)
sentences.shape

(87336, 31)

In [65]:
[bw_mapping[c] for c in list(sentences[102])]

['>',
 'F',
 'o',
 'r',
 ' ',
 'm',
 'o',
 'r',
 'e',
 ' ',
 'i',
 's',
 ' ',
 't',
 'o',
 ' ',
 'b',
 'e',
 ' ',
 's',
 'a',
 'i',
 'd',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 't',
 'o',
 ' ']

In [45]:

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

# From Aylien's blog https://github.com/AYLIEN/gan-intro/blob/master/gan.py
def linear(input, output_dim, scope=None, stddev=1.0):
    with tf.variable_scope(scope or 'linear'):
        with tf.variable_scope('weights'):
            w = tf.get_variable(
                'w',
                [input.get_shape()[1], output_dim]#,
                #initializer=tf.random_normal_initializer(stddev=stddev)
            )
            variable_summaries(w)
        with tf.variable_scope('bias'):
            b = tf.get_variable(
                'b',
                [output_dim]#,
                #initializer=tf.constant_initializer(0.5)
            )
            variable_summaries(b)
        return tf.matmul(input, w) + b, (w, b)

def bilinear(input, state, output_dim, scope=None, stddev=1.0):
    with tf.variable_scope(scope or 'linear'):
        with tf.variable_scope('input_weights'):
            iw = tf.get_variable(
                 'iw',
                 [input.get_shape()[1], output_dim]#,
                 #initializer=tf.random_normal_initializer(stddev=stddev)
            )
            variable_summaries(iw)
        with tf.variable_scope('state_weights'):
            sw = tf.get_variable(
                 'sw',
                 [state.get_shape()[1], output_dim]#,
                 #initializer=tf.random_normal_initializer(stddev=stddev)
            )
            variable_summaries(sw)
        with tf.variable_scope('bias'):
            b = tf.get_variable(
                'b',
                [output_dim]#,
                #initializer=tf.constant_initializer(0.5)
            )
            variable_summaries(b)
        return tf.matmul(input, iw) + tf.matmul(state, sw) + b, (iw, sw, b)

In [26]:
def my_optimize(sess, train, Xs, loss,
                name='train', steps=1000,
                optimizer=tf.train.GradientDescentOptimizer(0.5),
                dict={}, batch_size=100, report_steps=500):
    train_step = optimizer.minimize(loss)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('rnn_logs/%s' % name, sess.graph)
    
    for g, v in optimizer.compute_gradients(loss):
        #tf.summary.scalar("grad_{}".format(v.name), tf.norm(g))
        #merged = tf.Print(merged, [tf.norm(g)], message="grad_{}".format(v.name))
        pass
    
    sess.run(tf.global_variables_initializer())

    for i in range(steps):
        
        # Such a hack
        start = (i * batch_size) % train.shape[0]
        end = ((i+1) * batch_size) % train.shape[0]
        
        if start < end:
            batch_xs = train[start:end]
            dict = dict.copy()# Avoid getting the keys from a different graph
            dict[Xs] = batch_xs
            summary, _, xloss = sess.run([merged, train_step, loss], feed_dict=dict)
            
            if i % report_steps == 0:
                print('Step {}\tLoss: {}'.format(i, xloss))

            train_writer.add_summary(summary, i)
        
    train_writer.close()

In [52]:
hidden_state_size = 512

def rnn_text(sen_sz=31):

    init_state = tf.placeholder(tf.float32, [None, hidden_state_size])
    hidden_state = init_state

    seq_input = tf.placeholder(tf.int64, [None, sen_sz])
    enc_seq_input = tf.one_hot(seq_input, nletters) # adds a trailing dimension with the hot encoding

    costs = []
    
    for i in range(sen_sz-1):
        x = enc_seq_input[:, i, :]
        y = enc_seq_input[:, i+1, :]
        
        #x = tf.Print(x, [i, tf.argmax(tf.reshape(x, [nletters])), tf.argmax(tf.reshape(y, [nletters]))])
        
        with tf.variable_scope('step', reuse=i!=0):
            hidden_state, _ = bilinear(x, hidden_state, hidden_state_size, 'hl1')
            hidden_state = tf.nn.tanh(hidden_state)
            output, _ = linear(hidden_state, nletters, 'ol1')

        costs.append(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))
        
    loss = tf.reduce_mean(costs)
    
    return seq_input, init_state, loss

def rnn_synth_text(sen_sz=31):

    init_state = tf.placeholder(tf.float32, [None, hidden_state_size])
    hidden_state = init_state

    char_input = tf.placeholder(tf.int64, [None])
    x = tf.one_hot(char_input, nletters)
    
    with tf.variable_scope('step'):
        hidden_state, _ = bilinear(x, hidden_state, hidden_state_size, 'hl1')
        hidden_state = tf.nn.tanh(hidden_state)
        output, _ = linear(hidden_state, nletters, 'ol1')

    py = tf.nn.softmax(output)
    
    return char_input, init_state, py, hidden_state

In [71]:
batch_size = 256

g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        
        Xs, init_state, loss = rnn_text()
        
        optimizer = tf.train.AdamOptimizer()
        my_optimize(sess,
                    sentences,
                    Xs,
                    loss,
                    steps=3001,
                    report_steps=500,
                    name='train-1',
                    batch_size=batch_size,
                    dict={init_state: np.zeros((batch_size, hidden_state_size))},
                    optimizer=optimizer)
        
        tf.train.Saver().save(sess, 'artifacts/model-rnn-v2-1.ckpt')

Step 0	Loss: 4.440620422363281
Step 500	Loss: 2.117955446243286
Step 1000	Loss: 1.9799425601959229
Step 1500	Loss: 1.8005975484848022
Step 2000	Loss: 1.6870160102844238
Step 2500	Loss: 1.6520211696624756
Step 3000	Loss: 1.6153658628463745


In [72]:
batch_size = 1

g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        char_input, init_state, p_output, out_state = rnn_synth_text()
        
        tf.train.Saver().restore(sess, 'artifacts/model-rnn-v2-1.ckpt')
        #sess.run(tf.global_variables_initializer())
        
        res = ['>']
        cur_char = fw_mapping['>']
        cur_state = np.zeros((batch_size, hidden_state_size))

        for _ in range(500):#sen_sz - 1):        
            ps, state = sess.run([p_output, out_state],
                                 feed_dict={char_input: np.array([cur_char]), init_state: cur_state})
            #cur_char = np.argmax(ps)
            cur_char = np.random.choice(nletters, p=ps.reshape(nletters))
            cur_state = state
            res.append(bw_mapping[cur_char])
        
        print (''.join(res))


INFO:tensorflow:Restoring parameters from artifacts/model-rnn-v2-1.ckpt
>Call my tyou and tell her see is bedstries, 'Tis no demplous not quarren in my lond is she is saiding to dornous gread and neary inmmon's part, weak make pointed, here creeclest king tonesture as I dar, you did nighted; I despeting dore thy didilike cerague, ceede love, nor bigs from ancester! Where's no disters, revence to be frort'dlad on a mindred, I'll me tew proches, a will be mucted they cot love! I west aulied, I bester them, lords! O: when he to you actier pray that till Become, from hep


# Second part, using longer sentences

In [39]:
text = open('../data/shakespear.txt').read()

# mapping
letters = ['>'] + list(set(text))
nletters = len(letters)

bw_mapping = dict(enumerate(letters))
fw_mapping = dict([(c, ix) for ix, c in enumerate(letters)])

base_sz = 50
sen_sz = 1 + base_sz

# dataset
sentences = ['>' + text[i:i+base_sz] for i in range(0, len(text), base_sz)]
sentences = [[fw_mapping[c] for c in s] for s in sentences if len(s)==sen_sz]
sentences = np.array(sentences)

In [31]:
hidden_state_size = 512

def rnn_text(sen_sz=31):

    init_state = tf.placeholder(tf.float32, [None, hidden_state_size])
    hidden_state = init_state

    seq_input = tf.placeholder(tf.int64, [None, sen_sz])
    enc_seq_input = tf.one_hot(seq_input, nletters) # adds a trailing dimension with the hot encoding

    costs = []
    
    for i in range(sen_sz-1):
        x = enc_seq_input[:, i, :]
        y = enc_seq_input[:, i+1, :]
        
        #x = tf.Print(x, [i, tf.argmax(tf.reshape(x, [nletters])), tf.argmax(tf.reshape(y, [nletters]))])
        
        with tf.variable_scope('step', reuse=i!=0):
            hidden_state, _ = bilinear(x, hidden_state, hidden_state_size, 'hl1')
            hidden_state = tf.nn.tanh(hidden_state)
            output, _ = linear(hidden_state, nletters, 'ol1')

        costs.append(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))
        
    loss = tf.reduce_mean(costs)
    
    return seq_input, init_state, loss

def rnn_synth_text(sen_sz=31):

    init_state = tf.placeholder(tf.float32, [None, hidden_state_size])
    hidden_state = init_state

    char_input = tf.placeholder(tf.int64, [None])
    x = tf.one_hot(char_input, nletters)
    
    with tf.variable_scope('step'):
        hidden_state, _ = bilinear(x, hidden_state, hidden_state_size, 'hl1')
        hidden_state = tf.nn.tanh(hidden_state)
        output, _ = linear(hidden_state, nletters, 'ol1')

    py = tf.nn.softmax(output)
    
    return char_input, init_state, py, hidden_state

In [53]:
batch_size = 128

g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        
        Xs, init_state, loss = rnn_text(sen_sz=sen_sz)
        
        optimizer = tf.train.AdamOptimizer()
        my_optimize(sess,
                    sentences,
                    Xs,
                    loss,
                    steps=3001,
                    report_steps=500,
                    name='train-1',
                    batch_size=batch_size,
                    dict={init_state: np.zeros((batch_size, hidden_state_size))},
                    optimizer=optimizer)
        
        tf.train.Saver().save(sess, 'artifacts/model-rnn-shakespeare-200.ckpt')

Step 0	Loss: 4.191850662231445
Step 500	Loss: 2.0128650665283203
Step 1000	Loss: 1.5846800804138184
Step 1500	Loss: 1.3026831150054932
Step 2000	Loss: 1.0218708515167236
Step 2500	Loss: 0.8547126054763794
Step 3000	Loss: 0.7481168508529663


In [54]:
batch_size = 1

g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        char_input, init_state, p_output, out_state = rnn_synth_text(sen_sz=sen_sz)
        
        tf.train.Saver().restore(sess, 'artifacts/model-rnn-shakespeare-200.ckpt')
        
        res = ['>']
        cur_char = fw_mapping['>']
        cur_state = np.zeros((batch_size, hidden_state_size))

        for _ in range(500):#sen_sz - 1):        
            ps, state = sess.run([p_output, out_state],
                                 feed_dict={char_input: np.array([cur_char]), init_state: cur_state})
            #cur_char = np.argmax(ps)
            cur_char = np.random.choice(nletters, p=ps.reshape(nletters))
            cur_state = state
            res.append(bw_mapping[cur_char])
        
        print (''.join(res))


INFO:tensorflow:Restoring parameters from artifacts/model-rnn-shakespeare-200.ckpt
>ther thee
thal maillve mull rovech merness, tie pervait you f lleat you thel.

Th! Thoul
Tie thom at reance wet than se
Be ngain you tove: a will.

MINGA:
Pedisin ers reck ciptarus
no ertiof e wilt R
seles, and to, that fflones? hat yom seall, me tervaryous wat grom that
Is maknow tod'thingutrevill you, lyod I lave noll enveppreity und ler isanits
Ged Is mest you Ches lave me thaiflisea lave me tarese
me lids ay rivet

Kser llod is, st mens el furd ne, hid Sif lee boaks nos far whot wet he whe, 
